In [1]:
import phil_univax
import ast, itertools
import numpy as np
import concurrent.futures

In [2]:
opt_params_template = '''
{
'vaccine_total_avail[%(vaccine_id)d]' : '%(total_doses)d',
'vaccine_additional_per_day[%(vaccine_id)d]' : '%(daily_doses)d',                                                                                                      
'vaccine_dose_efficacy_values[%(vaccine_id)d][0]' : '1 %(efficacy)f',
}
'''

In [3]:
def parameterize(years, universal_coverage, universal_efficacy, seasonal_coverage, seasonal_efficacy):
    assert(years<=8)
    n = 1224000
    p = dict(days=360*years)
    p.update(ast.literal_eval(opt_params_template % dict(
        vaccine_id=0,
        total_doses=int(n*universal_coverage),
        daily_doses=int(n*universal_coverage/years/360),
        efficacy=universal_efficacy
    )))
    for vaccine_id in range(1,7):
        p.update(ast.literal_eval(opt_params_template % dict(
            vaccine_id=vaccine_id,
            total_doses=int(n*seasonal_coverage),
            daily_doses=int(n*seasonal_coverage),
            efficacy=seasonal_efficacy
        )))
    return p

In [4]:
universal_coverage_values = np.arange(0,0.5,0.1)
universal_efficacy_values = np.arange(0.1,0.5,0.1)
seasonal_coverage_values = np.arange(0.1,0.6,0.1)
seasonal_efficacy_values = np.arange(0.2,0.7,0.1)

In [5]:
reload(phil_univax)
pux = phil_univax.PhilUniversalExperiment()
results = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for (universal_coverage, universal_efficacy,
         seasonal_coverage, seasonal_efficacy) in itertools.product(
             universal_coverage_values, universal_efficacy_values,
             seasonal_coverage_values, seasonal_efficacy_values):
        opt_params = parameterize(3,
            universal_coverage, universal_efficacy,
            seasonal_coverage, seasonal_efficacy)
        run_name = '%f_%f_%f_%f' % (
            universal_coverage, universal_efficacy,
            seasonal_coverage, seasonal_efficacy)
        results[run_name] = executor.submit(pux.run, 'params.universal', opt_params, run_name)

In [16]:
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for k,v in results.items():
        if v.result() is False:
            (universal_coverage, universal_efficacy,
             seasonal_coverage, seasonal_efficacy) = [float(x) for x in k.split('_')]
            opt_params = parameterize(3,
                universal_coverage, universal_efficacy,
                seasonal_coverage, seasonal_efficacy)
            run_name = '%f_%f_%f_%f' % (
                universal_coverage, universal_efficacy,
                seasonal_coverage, seasonal_efficacy)
            results[run_name] = executor.submit(pux.run, 'params.universal', opt_params, run_name)        